In [1]:
import pandas as pd
import sqlite3

con = sqlite3.connect("sqlite_db_pythonsqlite.db")

In [2]:
query = "SELECT * FROM Facilities"
df = pd.read_sql_query(query, con)
print(df.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [3]:
query = "SELECT * FROM Members"
df = pd.read_sql_query(query, con)
print(df.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


In [4]:
query = "SELECT * FROM Bookings"
df = pd.read_sql_query(query, con)
print(df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [5]:
query = """
SELECT f.name,
    SUM(CASE WHEN b.memid=0 THEN b.slots*f.guestcost
        ELSE b.slots*f.membercost END) AS totalrev
FROM Bookings AS b
LEFT JOIN Facilities AS f
    ON b.facid = f.facid
LEFT JOIN Members AS m
    ON b.memid = m.memid
GROUP BY b.facid
HAVING totalrev < 1000
ORDER BY totalrev;
"""

df = pd.read_sql_query(query, con)
print(df)

            name  totalrev
0   Table Tennis       180
1  Snooker Table       240
2     Pool Table       270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [6]:
query = """
SELECT m1.surname || ', '|| m1.firstname AS member,
    m2.surname || ', ' || m2.firstname AS recommender
FROM Members as m1, Members as m2
WHERE m1.recommendedby = m2.memid
ORDER BY member;
"""

df = pd.read_sql_query(query, con)
print(df)

                      member         recommender
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

/* Q12: Find the facilities with their usage by member, but not guests */

In [7]:
query = """
SELECT f.name, SUM(slots) AS hours
FROM Bookings as b
LEFT JOIN Facilities AS f ON b.facid = f.facid
WHERE memid != 0
GROUP BY b.facid;
"""

df = pd.read_sql_query(query, con)
print(df)

              name  hours
0   Tennis Court 1    478
1   Tennis Court 2    441
2  Badminton Court    543
3     Table Tennis    397
4   Massage Room 1    442
5   Massage Room 2     27
6     Squash Court    209
7    Snooker Table    430
8       Pool Table    428


/* Q13: Find the facilities usage by month, but not guests */

In [9]:
query = """
SELECT f.name, STRFTIME('%m', starttime) AS month,
    SUM(slots)/2 AS hours
FROM Bookings as b
LEFT JOIN Facilities AS f ON b.facid = f.facid
WHERE memid != 0
GROUP BY b.facid, month;
"""

df = pd.read_sql_query(query, con)
print(df)

               name month  hours
0    Tennis Court 1    07    100
1    Tennis Court 1    08    169
2    Tennis Court 1    09    208
3    Tennis Court 2    07     61
4    Tennis Court 2    08    172
5    Tennis Court 2    09    207
6   Badminton Court    07     82
7   Badminton Court    08    207
8   Badminton Court    09    253
9      Table Tennis    07     49
10     Table Tennis    08    148
11     Table Tennis    09    200
12   Massage Room 1    07     83
13   Massage Room 1    08    158
14   Massage Room 1    09    201
15   Massage Room 2    07      4
16   Massage Room 2    08      9
17   Massage Room 2    09     14
18     Squash Court    07     25
19     Squash Court    08     92
20     Squash Court    09     92
21    Snooker Table    07     70
22    Snooker Table    08    158
23    Snooker Table    09    202
24       Pool Table    07     55
25       Pool Table    08    151
26       Pool Table    09    221


In [ ]:
con.close()